In [20]:
from urllib.parse import urlparse
import tldextract
from typing import Dict, Union
import requests
from bs4 import BeautifulSoup
import re

In [21]:


def evaluate_reference_credibility(url: str) -> Dict[str, Union[float, str]]:
    """
    Basic URL credibility evaluation based on domain type and known reliable sources.
    
    Args:
        url (str): The URL to evaluate
        
    Returns:
        dict: Contains 'score' (float) and 'explanation' (str)
    """
    try:
        # Parse the URL
        extracted = tldextract.extract(url)
        domain = f"{extracted.domain}.{extracted.suffix}"
        
        # Define trusted domains and their scores
        trusted_domains = {
            'edu': 0.9,    # Educational institutions
            'gov': 0.9,    # Government websites
            'org': 0.7,    # Non-profit organizations
        }
        
        # Define trusted sources
        trusted_sources = {
            'nature.com': 0.9,
            'science.org': 0.9,
            'scholar.google.com': 0.8,
        }
        
        # Basic scoring logic
        score = 0.5  # Default score
        explanation = []
        
        # Check domain type
        if extracted.suffix in trusted_domains:
            score = trusted_domains[extracted.suffix]
            explanation.append(f"Trusted domain type ({extracted.suffix})")
        
        # Check for trusted sources
        if domain in trusted_sources:
            score = trusted_sources[domain]
            explanation.append(f"Recognized trusted source ({domain})")
            
        return {
            "score": score,
            "explanation": " | ".join(explanation) if explanation else "Basic domain evaluation"
        }
        
    except Exception as e:
        return {
            "score": 0.0,
            "explanation": f"Error evaluating URL: {str(e)}"
        }

In [22]:
def evaluate_fact_check(url: str) -> Dict[str, Union[float, str]]:
    """
    Basic fact-checking evaluation based on known fact-checking sources and scientific journals.
    
    Args:
        url (str): The URL to evaluate
        
    Returns:
        dict: Contains 'fact_check_score' (float) and 'explanation' (str)
    """
    try:
        # Parse the URL
        parsed_url = urlparse(url)
        domain = parsed_url.netloc.lower()
        
        # Remove 'www.' if present
        if domain.startswith('www.'):
            domain = domain[4:]
            
        # Define fact-checking sources and their scores
        fact_check_sources = {
            'snopes.com': 0.9,
            'factcheck.org': 0.9,
            'politifact.com': 0.85,
            'reuters.com': 0.85,
            'apnews.com': 0.85,
            'nature.com': 0.95,       # Peer-reviewed scientific journal
            'science.org': 0.95,      # Peer-reviewed scientific journal
            'thelancet.com': 0.95,    # Medical journal
        }
        
        # Basic scoring logic
        score = 0.5  # Default score
        explanation = []
        
        # Check if it's a known fact-checking or scientific source
        for source, source_score in fact_check_sources.items():
            if domain == source or domain.endswith('.' + source):
                score = source_score
                if source in ['nature.com', 'science.org', 'thelancet.com']:
                    explanation.append(f"Peer-reviewed scientific source ({source})")
                else:
                    explanation.append(f"Recognized fact-checking source ({source})")
                break
                
        # Additional check for scientific article patterns
        path = parsed_url.path.lower()
        if any(x in path for x in ['/article/', '/research/', '/study/', '/paper/']):
            score += 0.1
            explanation.append("Contains scientific article indicators")
            score = min(score, 1.0)  # Cap at 1.0
            
        return {
            "fact_check_score": score,
            "explanation": " | ".join(explanation) if explanation else "Basic fact-check evaluation"
        }
        
    except Exception as e:
        return {
            "fact_check_score": 0.0,
            "explanation": f"Error evaluating URL: {str(e)}"
        }

In [23]:
def evaluate_citations(url: str) -> Dict[str, Union[float, str]]:
    """
    Evaluates citation count and reference quality from webpage content.
    
    Args:
        url (str): The URL to evaluate
        
    Returns:
        dict: Contains 'citation_score' (float) and 'explanation' (str)
    """
    try:
        # Fetch the webpage content
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        response = requests.get(url, headers=headers, timeout=10)
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Initialize counters and score
        citation_count = 0
        reference_count = 0
        score = 0.5  # Default score
        explanation = []
        
        # Find citations in different formats
        # Look for <cite> tags
        cite_tags = soup.find_all('cite')
        citation_count += len(cite_tags)
        
        # Look for numbered references [1], [2], etc.
        numbered_refs = re.findall(r'\[\d+\]', response.text)
        citation_count += len(numbered_refs)
        
        # Look for reference or bibliography section
        ref_sections = soup.find_all(['div', 'section'], 
                                   class_=re.compile(r'reference|bibliography|citations', re.I))
        
        if ref_sections:
            # Count references in these sections
            for section in ref_sections:
                # Count list items in reference sections
                references = section.find_all('li')
                reference_count += len(references)
        
        # Calculate score based on citations
        total_citations = max(citation_count, reference_count)
        
        if total_citations > 0:
            # Adjust score based on number of citations
            if total_citations >= 50:
                score = 0.9
            elif total_citations >= 30:
                score = 0.8
            elif total_citations >= 15:
                score = 0.7
            elif total_citations >= 5:
                score = 0.6
            
            explanation.append(f"Found {total_citations} citations/references")
            
        # Look for DOI references
        doi_refs = re.findall(r'doi\.org/\d+\.\d+/\S+', response.text)
        if doi_refs:
            score += 0.1  # Bonus for having DOI references
            explanation.append(f"Found {len(doi_refs)} DOI references")
            score = min(score, 1.0)  # Cap at 1.0
            
        return {
            "citation_score": score,
            "explanation": " | ".join(explanation) if explanation else "No citations found",
            "citation_count": total_citations
        }
        
    except Exception as e:
        return {
            "citation_score": 0.0,
            "explanation": f"Error analyzing citations: {str(e)}",
            "citation_count": 0
        }

In [24]:
"""
Main method to call evaluate_reference_credibility function and evaluate_reference_credibility and return the scores
"""

url = "https://www.nature.com/articles/s41586-020-2649-2"
result_domain = evaluate_reference_credibility(url)
result_fact_check = evaluate_fact_check(url)
result_citations = evaluate_citations(url)
print(result_domain)
print(result_fact_check)
print(result_citations)

{'score': 0.9, 'explanation': 'Recognized trusted source (nature.com)'}
{'fact_check_score': 0.95, 'explanation': 'Peer-reviewed scientific source (nature.com)'}
{'citation_score': 0.6, 'explanation': 'Found 2 citations/references | Found 3 DOI references', 'citation_count': 2}
